---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [10]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [101]:
#%ls
#%cat university_towns.txt
city_data = pd.read_csv('City_Zhvi_AllHomes.csv')
#city_data.columns.values
#city_data.loc[['RegionID']]


In [4]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 
              'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 
              'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia',
              'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington',
              'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey',
              'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico',
              'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota',
              'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut',
              'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas',
              'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California',
              'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island',
              'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia',
              'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
#import pandas as pd
#import re

In [4]:
#housing = pd.read_csv('City_Zhvi_AllHomes.csv')
#housing.head()

In [102]:
#list(housing.columns.values)

In [103]:
# clg_towns = pd.read_csv('university_towns.txt', sep = "\n", header=None)
# clg_towns.head()

In [5]:
#clg_towns[clg_towns[0].str.contains('\[')] #let's try the regex version

def mfunc(x):
    
    pattern = re.compile(r".*(\[[a-z]+\])")
    #pattern = re.compile(r"(\[[a-z]+\])") # for matching
    #match = re.findall(pattern, x)
    match = re.sub(pattern, " ", x)
    if match:
        return match
    else:
        return match
    

#clg_towns[0].map(mfunc)

In [12]:
#clg_towns[0][107] = "Illinois"

In [104]:
#[i for j in list(states.values()) for i in clg_towns[0] if i in j] #Doesn't work

In [14]:
# Use this dictionary to map state names to two letter acronyms
#states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [105]:
#'Alabama' in list(states.values())
#any("" in list(states.values()))
#([j for i,j in enumerate(clg_towns[0]) if j in [states.values()]])

In [6]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 
              'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 
              'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia',
              'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington',
              'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey',
              'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico',
              'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota',
              'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut',
              'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas',
              'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California',
              'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island',
              'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia',
              'ND': 'North Dakota', 'VA': 'Virginia'}
    
    clg_towns = pd.read_csv('university_towns.txt', sep = "\n", header=None)
       
    '''
    def filter_state(name):
        
        state = list()
        
        for i in name:
            return type(i)
            if any(i in ["", "Alabama"]):
            
                return "Hello"
        
        #return (name[2])
        return state
    
    x = clg_towns.apply(filter_state)
    
    '''

    def clean_names(x):
    
        #pattern = re.compile(r".*(\[[a-z]+\])") # for matching
        pattern = re.compile(r"(\[[a-z]*\])") # for substitution of state name
        pattern1 = re.compile(r"(\(.*)", flags = re.I) # for substitution of town name
        #match = re.findall(pattern, x)
        filter_name = re.sub(pattern, "", x)
        
        #return filter_name
    
        if filter_name in list(states.values()):
            
            ####filter_name = re.sub(pattern, "", x)
            #return filter_name
            result = filter_name.strip()
            return pd.Series({'State' : result})
        else:
            result = re.sub(pattern1, "", x).strip()
            #return town_name
            return pd.Series({'RegionName':result})
        
        
    x = clg_towns[0].apply(clean_names)
    
    x['State'] = x['State'].fillna(method = 'ffill')
    
    x = x.dropna(axis = 0)
    
    #x = x.groupby(list(states.values())).apply(lambda x: x)
    
    x = x[['State', 'RegionName']]
    x.reset_index(drop=True, inplace = True)
    
    return x

In [13]:
p = get_list_of_university_towns()
p
#p.describe()
#p.iloc[190:]#[p == "Michigan"]

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


### GDP data

In [110]:
gdp = pd.read_excel('gdplev.xls')
housing = pd.read_csv('City_Zhvi_AllHomes.csv')

In [113]:
gdp.drop('Unnamed: 3', axis = 1, inplace = True)

#gdp.drop('2016*', axis = 1, inplace = True)

gdp = gdp[gdp.columns.drop(list(gdp.filter(regex = '^[0-9]').astype('str')))]

In [114]:
gdp_yearly = gdp[[0,1,2]]
gdp_quarterly = gdp[[3,4,5]]

In [115]:
def clean_gdp(gdp_yearly):

    gdp_yearly = gdp_yearly.rename(columns = {gdp_yearly.columns.values[0]:gdp_yearly.iloc[[1],[0]].values[0][0],
                             gdp_yearly.columns.values[1]:gdp_yearly.iloc[[4],[1]].values[0][0],
                             gdp_yearly.columns.values[2]:gdp_yearly.iloc[[4],[2]].values[0][0]})
    gdp_yearly = gdp_yearly.drop(gdp_yearly.index[0:7]).reset_index(drop=True).dropna()
    
    return gdp_yearly

In [116]:
gdp_yearly = clean_gdp(gdp_yearly)

In [233]:
gdp_quarterly = clean_gdp(gdp_quarterly)

In [234]:
gdp_quarterly = gdp_quarterly.set_index('Quarterly').loc['2000q1':].reset_index()

In [235]:
gdp_quarterly.head()

,Quarterly,GDP in billions of current dollars,GDP in billions of chained 2009 dollars
0,2000q1,10031,12359.1
1,2000q2,10278.3,12592.5
2,2000q3,10357.4,12607.7
3,2000q4,10472.3,12679.3
4,2001q1,10508.1,12643.3


In [117]:
# off-track solution stopped working
def gdp_start(gdp):
    
    
    dollar_value = gdp.iloc[:, 2]
    recession_list = list()
    
    #return dollar_value.iloc[(len(dollar_value))-1]
    
    for i in range(1, len(dollar_value) - 1):
       
        if dollar_value.iloc[i-1] > dollar_value.iloc[i]:
            recession_list.append(dollar_value.iloc[i])
    #return dollar_value  
    return recession_list
    #return pd.DataFrame({'list': recession_list})
    

In [118]:
# testing and working
def test(gdp):
    gdp = gdp.set_index('Quarterly')
    
    dollar_value = gdp.iloc[:,1]
    
    recession_list = list()
    
    #return dollar_value.iloc[(len(dollar_value))-1]
    
    for i in range(1, len(dollar_value) - 1):
       
        if (dollar_value.iloc[i-1] > dollar_value.iloc[i]) & (dollar_value[i] > dollar_value[i+1]):
            return dollar_value.index[i-1]
            #recession_list.append((dollar_value.iloc[i-1], dollar_value.index[i-1]))
    #return dollar_value  
    return recession_list
    
            
    
test(gdp_quarterly)    
  

KeyError: 'Quarterly'

In [201]:
## off-track solution
#t = gdp_quarterly.set_index('Quarterly').iloc[:,1]
#t.iterrows()

In [207]:
# off-track that doesn't work
#gdp_quarterly.iloc[:, 2]
#gdp_quarterly.apply(gdp_start, axis = 1)

In [119]:
gdp_quarterly.groupby(gdp_quarterly['Quarterly'].str.extract(r'^([0-9]*)q[1-4]$')).apply(gdp_start)

KeyError: 'Quarterly'

In [90]:
gdp_quarterly.columns[1]

'GDP in billions of current dollars'

In [120]:
def get_recession_start():
    
      
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    
    gdp = pd.read_excel('gdplev.xls')
    
    gdp.drop('Unnamed: 3', axis = 1, inplace = True)

    #gdp.drop('2016*', axis = 1, inplace = True)

    gdp = gdp[gdp.columns.drop(list(gdp.filter(regex = '^[0-9]').astype('str')))]
    
    gdp_yearly = gdp[[0,1,2]]
    gdp_quarterly = gdp[[3,4,5]]
    
    def clean_gdp(gdp_yearly):

        gdp_yearly = gdp_yearly.rename(columns = {gdp_yearly.columns.values[0]:gdp_yearly.iloc[[1],[0]].values[0][0],
                             gdp_yearly.columns.values[1]:gdp_yearly.iloc[[4],[1]].values[0][0],
                             gdp_yearly.columns.values[2]:gdp_yearly.iloc[[4],[2]].values[0][0]})
        gdp_yearly = gdp_yearly.drop(gdp_yearly.index[0:7]).reset_index(drop=True).dropna()
    
        return gdp_yearly

    gdp_yearly = clean_gdp(gdp_yearly)
    
    gdp_quarterly = clean_gdp(gdp_quarterly)
    
    #return gdp_quarterly.set_index('Quarterly')
    
    gdp_quarterly = gdp_quarterly.set_index('Quarterly').loc['2000q1':].reset_index()
    
    
    def recession_string(gdp):
        gdp = gdp.set_index('Quarterly')
    
        dollar_value = gdp.iloc[:,1]
    
        recession_list = list()
    
        #return dollar_value.iloc[(len(dollar_value))-1]
    
        for i in range(1, len(dollar_value) - 1):
       
            if (dollar_value.iloc[i-1] > dollar_value.iloc[i]) & (dollar_value[i] > dollar_value[i+1]):
                #return dollar_value.index[i-1]
                recession_list.append((dollar_value.iloc[i], dollar_value.index[i]))
        #return dollar_value  
        return recession_list[0][1]
    
    return recession_string(gdp_quarterly)
    

In [121]:
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    
    gdp = pd.read_excel('gdplev.xls')
    
    gdp.drop('Unnamed: 3', axis = 1, inplace = True)

    #gdp.drop('2016*', axis = 1, inplace = True)

    gdp = gdp[gdp.columns.drop(list(gdp.filter(regex = '^[0-9]').astype('str')))]
    
    gdp_yearly = gdp[[0,1,2]]
    gdp_quarterly = gdp[[3,4,5]]
    
    def clean_gdp(gdp_yearly):

        gdp_yearly = gdp_yearly.rename(columns = {gdp_yearly.columns.values[0]:gdp_yearly.iloc[[1],[0]].values[0][0],
                             gdp_yearly.columns.values[1]:gdp_yearly.iloc[[4],[1]].values[0][0],
                             gdp_yearly.columns.values[2]:gdp_yearly.iloc[[4],[2]].values[0][0]})
        gdp_yearly = gdp_yearly.drop(gdp_yearly.index[0:7]).reset_index(drop=True).dropna()
    
        return gdp_yearly

    gdp_yearly = clean_gdp(gdp_yearly)
    
    gdp_quarterly = clean_gdp(gdp_quarterly)
    
    #return gdp_quarterly.set_index('Quarterly')
    
    gdp_quarterly = gdp_quarterly.set_index('Quarterly').loc['2000q1':].reset_index()
    
    
    def recession_st_string(gdp):
        gdp = gdp.set_index('Quarterly')
    
        dollar_value = gdp.iloc[:,1]
    
        recession_list = list()
    
        #return dollar_value.iloc[(len(dollar_value))-1]
    
        for i in range(1, len(dollar_value) - 1):
       
            if (dollar_value.iloc[i-1] > dollar_value.iloc[i]) & (dollar_value[i] > dollar_value[i+1]):
                #return dollar_value.index[i-1]
                recession_list.append((dollar_value.iloc[i], dollar_value.index[i]))
        #return dollar_value  
        return recession_list[0][1]
    
    
    recession_start = recession_st_string(gdp_quarterly)
    
    ''' Recession end '''
    
    def recession_end_string(gdp, recession_start):
        gdp = gdp.set_index('Quarterly')
        gdp = gdp.loc[recession_start:].iloc[1:]
        
        #return gdp
        
        dollar_value = gdp.iloc[:, 1]
        
        recession_list = list()
        
        for i in range(1, len(dollar_value) - 1):
            
            if(dollar_value.iloc[i-1] < dollar_value.iloc[i]) & (dollar_value.iloc[i] < dollar_value.iloc[i+1]):
                recession_list.append((dollar_value.iloc[i+1], dollar_value.index[i+1]))
                
        
        return recession_list[0][1]
        
    
    return recession_end_string(gdp_quarterly, recession_start)

In [4]:
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    
    gdp = pd.read_excel('gdplev.xls')
    
    gdp.drop('Unnamed: 3', axis = 1, inplace = True)

    #gdp.drop('2016*', axis = 1, inplace = True)

    gdp = gdp[gdp.columns.drop(list(gdp.filter(regex = '^[0-9]').astype('str')))]
    
    gdp_yearly = gdp[[0,1,2]]
    gdp_quarterly = gdp[[3,4,5]]
    
    def clean_gdp(gdp_yearly):

        gdp_yearly = gdp_yearly.rename(columns = {gdp_yearly.columns.values[0]:gdp_yearly.iloc[[1],[0]].values[0][0],
                             gdp_yearly.columns.values[1]:gdp_yearly.iloc[[4],[1]].values[0][0],
                             gdp_yearly.columns.values[2]:gdp_yearly.iloc[[4],[2]].values[0][0]})
        gdp_yearly = gdp_yearly.drop(gdp_yearly.index[0:7]).reset_index(drop=True).dropna()
    
        return gdp_yearly

    gdp_yearly = clean_gdp(gdp_yearly)
    
    gdp_quarterly = clean_gdp(gdp_quarterly)
    
    #return gdp_quarterly.set_index('Quarterly')
    
    gdp_quarterly = gdp_quarterly.set_index('Quarterly').loc['2000q1':].reset_index()
    
    
    def recession_st_string(gdp):
        gdp = gdp.set_index('Quarterly')
    
        dollar_value = gdp.iloc[:,1]
    
        recession_list = list()
    
        #return dollar_value.iloc[(len(dollar_value))-1]
    
        for i in range(1, len(dollar_value) - 1):
       
            if (dollar_value.iloc[i-1] > dollar_value.iloc[i]) & (dollar_value[i] > dollar_value[i+1]):
                #return dollar_value.index[i-1]
                recession_list.append((dollar_value.iloc[i], dollar_value.index[i]))
        #return dollar_value  
        return recession_list[0][1]
    
    
    recession_start = recession_st_string(gdp_quarterly)
    
    ''' Recession end '''
    
    def recession_end_string(gdp, recession_start):
        gdp = gdp.set_index('Quarterly')
        gdp = gdp.loc[recession_start:].iloc[1:]
        
        #return gdp
        
        dollar_value = gdp.iloc[:, 1]
        
        recession_list = list()
        
        for i in range(1, len(dollar_value) - 1):
            
            if(dollar_value.iloc[i-1] < dollar_value.iloc[i]) & (dollar_value.iloc[i] < dollar_value.iloc[i+1]):
                recession_list.append((dollar_value.iloc[i+1], dollar_value.index[i+1]))
                
        
        return recession_list[0][1]
    
    recession_end = recession_end_string(gdp_quarterly, recession_start)
    
    def recession_bottom(gdp, start, end):
        gdp = gdp.set_index('Quarterly')
        gdp = gdp.loc[start:end]
        
        recession_list = list()
        
        dollar_value = gdp.iloc[:, 1]
        
        return dollar_value.idxmin()
        
        
    
    return recession_bottom(gdp_quarterly, recession_start, recession_end)
    
    

In [7]:
get_recession_bottom()

'2009q2'

In [122]:
def old_convert_data_to_quarters_1():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    '''
    def my_func(data):
        
        #return type(data)
        
        desired = [('RegionID', 'SizeRank'), ('2000-01', '2016-08')]
        
        quarter_map = {'01': 'q1', '02': 'q1', '03': 'q1', '04': 'q2',
                  '05': 'q2', '06': 'q2', '07': 'q3', '08': 'q3',
                  '09': 'q3', '10':'q4', '11': 'q4', '12': 'q4'}

        t = 0
        count = 0
        for i in range(0, len(data)):
            
            t += (data[i])
            count += 1
        
        return (t/count)
        
    '''    
    
    city_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    ##### THE SHORTEST!
    city_data = city_data.filter(regex=("^(?!199.+)"))
    
    #return city_data.iloc[:,np.r_[0:6, 51:67]]
    
    ###'''
    #desired = [('RegionID', 'SizeRank'), ('2000-01', '2016-08')]
    
    #temp = pd.DataFrame()
    
    #for (x,y) in desired:
        #start = city_data.columns.get_loc(x)
        #end = city_data.columns.get_loc(y)
        #for i in range(start, end+1):
            
            #selection = city_data.iloc[:, i]
            
            #temp = temp.append(selection)
    
    #city_data_split = temp.transpose()
    #return city_data_split
    ###'''
    
    #city_data_split = pd.concat([city_data.iloc[:, :city_data.columns.get_loc("SizeRank")], city_data.iloc[city_data.columns.get_loc("2000-01"):]], axis=1) 
    
    #return city_data_split

    #return city_data_split.apply(my_func, axis = 0)
    #flag = 0
    quarter_map = {'01': 'q1', '02': 'q1', '03': 'q1', '04': 'q2',
                  '05': 'q2', '06': 'q2', '07': 'q3', '08': 'q3',
                  '09': 'q3', '10':'q4', '11': 'q4', '12': 'q4'}
    
    new_df = pd.DataFrame()
    col_start_index = city_data.columns.get_loc("SizeRank")
    col_end_index = len(city_data.columns)
    total_rows = len(city_data)
    
    for j in range(0, total_rows):
        
        m_dict = dict()
        #m_list = []
        
        for i in range(col_start_index + 1, col_end_index):
            
            #if flag == 1:
                #if city_data.columns[i].str
            #return i
            #if city_data.columns[i]
            col_name = city_data.columns[i]
           
            year_str = re.findall('(\d\d\d\d)\-\d\d|$', col_name)[0]
            month_str = re.findall('\d\d\d\d\-(\d\d)|$', col_name)[0]
            
            #return (city_data.iloc[j, i])
            #return np.isnan(city_data.iloc[j, i])
            #if np.isnan(city_data.iloc[j, i]) == True:
                #price = 0
            cell_value = city_data.iloc[j, i]     
            price = (cell_value) if ~np.isnan(cell_value) else 0
            
            if month_str and year_str:
                month = int(month_str)
                year = int(year_str)
                quarter = quarter_map[(month_str)]
                # return type(int(month_str))
               
            
                # if (year_str+quarter) in m_dict:
                if (int(month_str) % 3) == 1:
                    #return m_dict
                    #return m_dict[year_str]
                    # pass
                    m_dict[year_str+quarter] = [price]
                    #count += 1          

                else:
                    m_dict[year_str+quarter].append(price)
                    #count = 1
                    #return m_dict
            
            
        temp_df = pd.DataFrame([m_dict])
        new_df = pd.concat([new_df, temp_df], axis = 0, ignore_index = True)
        
        #if j == 100:    
    return new_df 
                 
            #elif city_data.columns[i] == "SizeRank":
                #flag = 1
            #else:         
            # do something with the columns
    
    return city_data.apply(my_func, axis = 1)
    #return city_data_split.apply(my_func)
    
    
    
    return "ANSWER"

In [185]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    '''
    def convert_to_quarters(m_list):
        
        for i in range(0, len(m_list)):
            month_str = re.findall('\d\d\d\d\-(\d\d)|$', m_list[0])[0]
            
        return len(m_list)
    '''
    
    def new_col_names(quarter_map, columns):
        
        new_cols = list()
        
        for x in columns:
            year_str = re.findall('(\d\d\d\d)\-\d\d|$', x)[0]
            month_str = re.findall('\d\d\d\d\-(\d\d)|$', x)[0]
            quarter = quarter_map[(month_str)]
            
            if (year_str+quarter) in new_cols:
                pass
            else:
                new_cols.append(year_str+quarter)
        return new_cols
        
    #generating the new coloumns names 
        
    city_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    city_data['State'] = city_data['State'].map(states)
    city_data = city_data.set_index(['State', 'RegionName'])
    
    ##### THE SHORTEST!
    #city_data = city_data.filter(regex=("^(?!199.+)"))
    
    city_data = city_data.filter(regex=("^(20.+)"))
   
    quarter_map = {'01': 'q1', '02': 'q1', '03': 'q1', '04': 'q2',
                  '05': 'q2', '06': 'q2', '07': 'q3', '08': 'q3',
                  '09': 'q3', '10':'q4', '11': 'q4', '12': 'q4'}
    
    new_df = pd.DataFrame()
    
    packed_columns = [city_data.columns[x:x+3] for x in range(0, len(city_data.columns), 3)]
    
    #return list(map(convert_to_quarters, packed_columns))
    
    #for x in packed_columns:
        #new_df = pd.concat([new_df, city_data[x]], axis = 0)
    #return new_df

    column_names = new_col_names(quarter_map, city_data.columns)
    
    for col, q in zip(column_names, packed_columns):
        
        new_df[col] = city_data[q].mean(axis = 1)
        
    return new_df

In [203]:
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [124]:
#convert_housing_data_to_quarters()

In [125]:
#%%time
#convert_housing_data_to_quarters()

In [126]:
#convert_housing_data_to_quarters()

In [127]:
#convert_housing_data_to_quarters()

In [281]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    
    uni_towns = get_list_of_university_towns()
    housing_data_quarters = convert_housing_data_to_quarters()
    new_data = pd.DataFrame()
    
    housing_recession = housing_data_quarters.iloc[:, (housing_data_quarters.columns.get_loc("2008q3")):(housing_data_quarters.columns.get_loc("2009q2"))+1]
    
    housing_recession = housing_recession.apply(lambda x: x[3] - x[0], axis = 1)
    
    university_group = uni_towns.set_index(["State", "RegionName"])
    
    # the result below seems valid because not all university towns are necessarily in the housing data
    university_group = housing_recession[housing_recession.index.isin(university_group.index)]
    
    # note down
    non_university_group = housing_recession[~housing_recession.index.isin(university_group.index)]
    
    t1 = pd.DataFrame({'value': university_group.values})
    t2 = pd.DataFrame({'value': non_university_group.values})

    
    better = ("university town" if (t1.mean().value > t2.mean().value) else "non-university town")
    
    # note down
    t_test = ttest_ind(t1['value'].dropna(), t2['value'].dropna(), equal_var=False)
    
    different = (True if (t_test.pvalue < 0.01) else False)
    
    return (different, t_test.pvalue, better)


In [282]:
run_ttest()

(True, 6.7029222130854317e-05, 'university town')

In [244]:
run_ttest()

[            value
 0   -49033.333333
 1   -35166.666667
 2    -7066.666667
 3    -7033.333333
 4    -2433.333333
 5    -7700.000000
 6   -22133.333333
 7    -4200.000000
 8   -11166.666667
 9   -11400.000000
 10  -33466.666667
 11  -17300.000000
 12   -1333.333333
 13  -21100.000000
 14  -89033.333333
 15   -1833.333333
 16  -24566.666667
 17   -9466.666667
 18   -1400.000000
 19   11266.666667
 20   -4600.000000
 21  -24366.666667
 22   -6266.666667
 23   -1533.333333
 24  -10300.000000
 25   -2966.666667
 26    5900.000000
 27    -733.333333
 28   -5066.666667
 29  -44033.333333
 ..            ...
 239  -1333.333333
 240 -13933.333333
 241 -22133.333333
 242 -11333.333333
 243   4033.333333
 244  -3266.666667
 245  -1233.333333
 246  -5566.666667
 247 -20233.333333
 248   3733.333333
 249  -1066.666667
 250  -1233.333333
 251  -1066.666667
 252   1833.333333
 253 -19366.666667
 254  -1266.666667
 255   4866.666667
 256   -333.333333
 257    216.666667
 258  -6000.000000
 259        